In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('0709_combined_data_topcities.csv')

In [3]:
df

,sentence,label,intent
0,round trip fares Warsaw to Istanbul,O O O origin O destination,other
1,if I am going to Berlin should I bring umbrella,O O O O O location O O O O,weather
2,what is the ground transportation from philade...,O O O O O O O O O O O O,other
3,okay and on may four i would like to go from S...,O O O O O O O O O O O origin O destination O O...,flight
4,do I need to bring umbrella to Seoul today,O O O O O O O location O,weather
5,temperature in Rome,O O location,weather
6,current weather in Florence,O O O location,weather
7,what 's the cheapest round trip flight on twa ...,O O O O O O O O O O origin O destination,flight
8,will Los Angeles be hot,O location location O O,weather
9,I want to know if it will rain in Buenos Aires,O O O O O O O O O location location,weather


In [4]:
data_arr = df.values

In [5]:
s_list, l_list, i_list = [],[],[]

In [6]:
for row in data_arr:
    s_list.append(row[0])
    l_list.append(row[1])
    i_list.append(row[2])

[len(x) for x in [s_list,l_list,i_list]]

[3600, 3600, 3600]

In [7]:
s_list

['round trip fares Warsaw to Istanbul',
 'if I am going to Berlin should I bring umbrella',
 'what is the ground transportation from philadelphia airport to the city proper',
 'okay and on may four i would like to go from Sydney to Chennai leaving early in the morning around _D_',
 'do I need to bring umbrella to Seoul today',
 'temperature in Rome',
 'current weather in Florence',
 "what 's the cheapest round trip flight on twa from Burgas to Kraków",
 'will Los Angeles be hot',
 'I want to know if it will rain in Buenos Aires',
 'Doha weather',
 'do I need to bring umbrella to Melbourne today',
 'i want to know the cheapest way to fly from Milan to Honolulu',
 'list all flights from Buenos Aires to Kraków on friday',
 'show me the flights that go from Bucharest to Seoul on thursday mornings',
 "i 'd like to fly from Prague to Brussels on american airlines",
 'what airline is ac',
 'what are the fares for flights between Tokyo and Burgas dc',
 "all right what i 'd like to do is find t

In [8]:
set(x for y in l_list for x in y.split())

{'O', 'destination', 'location', 'origin'}

In [9]:
set(x for x in i_list)

{'flight', 'other', 'weather'}

In [10]:
# encode l_list
# encode index = 0 for l_list, to be used for padding
# prepare dictionary

l_dict = {}
l_dict['_unknown'] = 0
for index, label in enumerate(set(x for y in l_list for x in y.split()),1):
    l_dict[label] = index
    
print(l_dict)

{'_unknown': 0, 'destination': 1, 'O': 2, 'origin': 3, 'location': 4}


In [11]:
# pad all the labels to a length of 50

encoded_l = np.zeros((len(l_list),50,len(l_dict)))

# iterate through the l_list

for row_index, row in enumerate(l_list):
    
    l = row.split()
    
    for num in range(50):
        if num < len(l):
            encoded_l[row_index][num][l_dict[l[num]]] = 1
        else:
            encoded_l[row_index][num][0] = 1

In [12]:
encoded_l.shape

(3600, 50, 5)

In [13]:
# encode i_list
# prepare dictionary

i_dict = {}
for index, label in enumerate(set(x for y in i_list for x in y.split())):
    i_dict[label] = index
    
print(i_dict)

{'flight': 0, 'other': 1, 'weather': 2}


In [14]:
encoded_i = np.zeros((len(l_list),len(i_dict)))

# iterate through the i_list

for intent_index, intent in enumerate(i_list):
    encoded_i[intent_index][i_dict[intent]] = 1

In [15]:
encoded_i.shape

(3600, 3)

In [16]:
# use GloVe word embeddings for s_list

from gensim.models import KeyedVectors

C:\Users\ZYCHE\Anaconda2\envs\Python (DL)\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [17]:
model = KeyedVectors.load_word2vec_format('glove-wiki-gigaword-300/glove-wiki-gigaword-300.txt')

In [18]:
embedded_s = np.zeros((len(s_list),50,300))

# iterate through the s_list

for s_index, s in enumerate(s_list):
    words = s.split()
    
    for num in range(50):
        if num < len(words):
            try: 
                embedded_s[s_index][num] = model.wv.get_vector(words[num].lower())
            except KeyError:
                
                print(words[num].lower(),'not in vocab')
                embedded_s[s_index][num] = model.wv.get_vector('unk')

C:\Users\ZYCHE\Anaconda2\envs\Python (DL)\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ZYCHE\Anaconda2\envs\Python (DL)\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


_d_ not in vocab
_d_ not in vocab
ap_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
ap_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
m_d_ not in vocab
_d_ not in vocab
ap_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
nationair not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
nationair not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in vocab
_d_ not in v

In [19]:
embedded_s.shape

(3600, 50, 300)

In [20]:
encoded_l.shape

(3600, 50, 5)

In [21]:
len(i_list)

3600

In [22]:
intent_target = np.zeros((len(i_list),3))

intent_target.shape

(3600, 3)

In [23]:
intent_target[0]

array([0., 0., 0.])

In [24]:
# 0 = flight, 1 = weather, 2 = other

for index, i in enumerate(i_list):
    if i == 'flight':
        intent_target[index] = [1,0,0]
    elif i == 'weather':
        intent_target[index] = [0,1,0]
    else:
        intent_target[index] = [0,0,1]

In [25]:
intent_target

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [26]:
i_list

['other',
 'weather',
 'other',
 'flight',
 'weather',
 'weather',
 'weather',
 'flight',
 'weather',
 'weather',
 'weather',
 'weather',
 'flight',
 'flight',
 'flight',
 'flight',
 'other',
 'other',
 'other',
 'weather',
 'other',
 'flight',
 'other',
 'flight',
 'weather',
 'weather',
 'other',
 'other',
 'flight',
 'flight',
 'flight',
 'flight',
 'other',
 'weather',
 'other',
 'flight',
 'flight',
 'other',
 'weather',
 'other',
 'weather',
 'flight',
 'flight',
 'other',
 'other',
 'weather',
 'weather',
 'weather',
 'other',
 'weather',
 'weather',
 'flight',
 'flight',
 'weather',
 'weather',
 'flight',
 'weather',
 'flight',
 'other',
 'weather',
 'flight',
 'weather',
 'other',
 'other',
 'flight',
 'flight',
 'weather',
 'weather',
 'flight',
 'weather',
 'flight',
 'flight',
 'other',
 'other',
 'flight',
 'other',
 'flight',
 'flight',
 'other',
 'weather',
 'other',
 'weather',
 'flight',
 'weather',
 'weather',
 'weather',
 'other',
 'flight',
 'flight',
 'weather',
 '

In [27]:
from keras.models import Model
from keras.regularizers import l2
from keras.layers import Input, Dense, LSTM, Dropout, Conv1D, MaxPool1D, Flatten, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.layers.merge import concatenate
from keras.utils import plot_model

Using TensorFlow backend.


In [34]:
visible = Input(shape=(50,300))
hidden1 = Dropout(0.2)(visible)
hidden2 = LSTM(100,return_sequences=True)(hidden1)

# CNN layers 
c_hidden1 = Conv1D(filters=50,kernel_size=5)(hidden2)
c_hidden2 = MaxPool1D(pool_size=5)(c_hidden1)

c_extract = Reshape((50,9))(c_hidden2)


# CNN layers 2

c2_hidden1 = Conv1D(filters=50,kernel_size=2)(hidden2)
c2_extract = Reshape((50,49))(c2_hidden1)

# Time distributed layers for labelling
t_hidden1 = Dense(64,activation='relu',kernel_regularizer=l2(0.01))(hidden2)
t_hidden2 = Dropout(0.2)(t_hidden1)
t_hidden3 = concatenate([c_extract,c2_extract,t_hidden2])
t_hidden4 = Flatten()(t_hidden3)
t_hidden5 = Dense(16, activation='relu')(t_hidden4)
t_output = Dense(3,activation='softmax')(t_hidden5)

pred_model = Model(inputs=visible,outputs=t_output)

In [35]:
pred_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50, 300)      0                                            
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 50, 300)      0           input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 50, 100)      160400      dropout_4[0][0]                  
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 46, 50)       25050       lstm_3[0][0]                     
__________________________________________________________________________________________________
max_poolin

In [36]:
pred_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [37]:
pred_model.fit(x=embedded_s,y=intent_target,epochs=30,batch_size=64,validation_split=0.3)

Train on 2520 samples, validate on 1080 samples
Epoch 1/30
2520/2520 [==============================] - 23s 9ms/step - loss: 1.0456 - acc: 0.7873 - val_loss: 0.6235 - val_acc: 0.9194
Epoch 2/30
2520/2520 [==============================] - 7s 3ms/step - loss: 0.4587 - acc: 0.9433 - val_loss: 0.3351 - val_acc: 0.9583
Epoch 3/30
2520/2520 [==============================] - 7s 3ms/step - loss: 0.2658 - acc: 0.9627 - val_loss: 0.2058 - val_acc: 0.9759
Epoch 4/30
2520/2520 [==============================] - 7s 3ms/step - loss: 0.1746 - acc: 0.9702 - val_loss: 0.1418 - val_acc: 0.9769
Epoch 5/30
2520/2520 [==============================] - 7s 3ms/step - loss: 0.1009 - acc: 0.9770 - val_loss: 0.1084 - val_acc: 0.9787
Epoch 6/30
2520/2520 [==============================] - 7s 3ms/step - loss: 0.0674 - acc: 0.9841 - val_loss: 0.0714 - val_acc: 0.9870
Epoch 7/30
2520/2520 [==============================] - 7s 3ms/step - loss: 0.0546 - acc: 0.9837 - val_loss: 0.0727 - val_acc: 0.9833
Epoch 8/30
25

In [42]:
intent_dict = {0:'flight',1:'weather',2:'other'}

In [43]:
def input_to_intent():
    x = input().strip().lower()
    
    words = x.split()
    
    embedded_input = np.zeros((1,50,300))
    
    for index, word in enumerate(words):
        try:
            embedded_input[0][index] = model.get_vector(word)
        except KeyError:
            embedded_input[0][index] = model.get_vector('unk')
    
    outputs = pred_model.predict(embedded_input)
    
    intent = intent_dict[np.argmax(outputs[0])]
    
    return intent

In [45]:
input_to_intent()

hot or not


'weather'

In [47]:
input_to_intent()

ticket to singapore


'other'

In [48]:
input_to_intent()

fly to malaysia


'flight'

In [49]:
input_to_intent()

fly


'flight'

In [50]:
input_to_intent()

to malaysia


'flight'

In [51]:
input_to_intent()

to somewhere


'flight'

In [52]:
input_to_intent()

to


'other'

In [53]:
input_to_intent()

somewhere


'weather'

In [54]:
input_to_intent()

temp in singapore


'weather'

In [57]:
input_to_intent()

hot singapore


'weather'

In [58]:
input_to_intent()

hot city


'weather'

In [59]:
input_to_intent()

hot here or not


'weather'

In [60]:
input_to_intent()

snow in japan?


'weather'

In [61]:
input_to_intent()

flight to singapore


'flight'

In [62]:
pred_model.save('lstm_intent.h5')

In [63]:
pred_model.save_weights('lstm_intent_weights.h5')